In [ ]:
#UVOD
Pri predmetu Podatkovno rudarjenje smo dobili nalogo, da si izberemo zanimiv in kompleksen problem, na katerem bo temeljila naša projektna naloga. Izbrali smo si podatke o tehničnih pregledih in se odločili poglobiti v problem opravljanja tehničnih pregledov vozil z različnimi lastnostmi. V nadaljevanju bomo ta problem predstavili in z analizo podatkov poskušali odgovoriti na čim več zastavljenih vprašanj.


#CILJI NALOGE

Zadali smo si naslednje cilje:


- Z analizo rezultatov tehničnih pregledov motornih vozil bomo ugotovili, katera od lastnosti vozila (znamka, starost, prevoženi kilometri itd.) najbolj vpliva na verjetnost, da vozilo uspešno opravi tehnični pregled.

- Lastnosti, ki najbolj vplivajo na uspešnost tehničnih pregledov bomo bolj podrobno analizirali, npr. katera znamka je na tehničnih pregledih najmanj zanesljiva, ali imajo novejši avtomobili več možnosti, da uspešno opravijo tehnični pregled itd.

- Ugotovili bomo, ali se število tehničnih pregledov z leti zvišuje in kako se ob tem spreminja odstotek uspešnosti pregledov.

- Ugotovili bomo, kako se po letih spreminja število uvoženih avtomobilov in če imajo večjo verjetnost, da opravijo tehnični pregled.

- Bolj podrobno bomo po letih analizirali prevožene kilometre posameznih vozil in določili katerim vozilom so lastniki kilometre prevrteli nazaj. Nato bomo izračunali odstotek vozil s prevrtenimi kilometri na splošno in nato še v primerjavi z drugimi atributi npr. katera znamka vozila je bila največkrat prevrtena.

- Izračunali bomo, koliko avtomobilov je bilo izvoženih v zadnjih petih letih. Ugotovili bomo tudi katera znamka je bila najbolj popularna za izvoz.

# PODATKI

Glavni vir podatkov bo spletna stran ministrstva za javno upravo (vir: https://podatki.gov.si/) in sicer se bomo osredotočili na rubriko "Promet in infrastruktura", kjer je veliko uporabnih skupin podatkov, med katerimi se bomo osredotočili na skupino, ki opisuje rezultate tehničnih pregledov motornih vozil, saj vsebuje veliko podrobnosti o različnih avtomobilih in postopkih tehničnega pregleda (npr. znamka avta, model avta, fizična/pravna oseba, število prevoženih km, leto prve registracije, podrobnosti v homologaciji, ipd.). Podatke bomo na koncu probali predstaviti na zanimiv in poučen način s pomočjo pestrih grafov.

# ZAČETKI NALOGE

Z analizo podatkov smo začeli z branjem vseh podatkov v python. Odločili smo se, da bomo uporabili podatke zadnjih 5-ih let. Že tu smo naleteli na težave, ker podatki čez leta niso bili poenoteni npr. med leti so bile različne datoteke (.csv, .txt), zaradi količine podatkov, so kasnejših letih bili podatki razpršeni v več datotek, medtem ko so bili podatki v bolj zgodnjih letih v enih datotekah, zato smo bili primorani podatke standardizirati. Tu pa smo tudi naleteli na težave, saj so bile nekatere združene datoteke prevelike, da bi jih python uspešno prebral. Ko smo to rešili, pa smo ugotovili, da nekatere datoteke uporabljajo ANSI kodiranje, nekatere pa UTF-8, to smo popravili tako, da smo pri branju datotek za vsako branje določili pravilno kodiranje.
Po vseh težavah smo končno lahko prebrali datoteke in jih shranili v imenik, ki je za ključe uporabljal leto iz katerega so bili podatki prebrani. Vrednost imenikov je tabela treh pandas tabel. V prvi pandas tabeli so podatki o neuspelih tehničnih pregledih, v drugi tabeli so podatki o vseh tehničnih pregledih tistega leta, v tretji tabeli pa so shranjene vsi podatki o meritvah.


#IZPOLNJEVANJE CILJEV

## KORELACIJA POSAMEZNIH ATRIBUTOV Z USPEŠNOSTJO TEHNIČNEGA PREGLEDA
Najprej smo za vsak atribut izračunali korelacijo glede na neuspešno opravljen tehnični pregled in jih s pridobljenimi podatki razporedili po vrsti. Kot pričakovano so prevoženi kilometri najbolj vplivali na izid tehničnega pregleda.


Slika prikazuje izpis atributov, ki najbolj vplivajo na izid tehničnega pregleda.

## PODROBNA ANALIZA NAJPOMEMBNEJŠIH ATRIBUTOV
Za bolj podrobno analizo smo si izbrali atributa prevoženi kilometri in znamka avtomobila. Za oba smo naredili graf, ki prikazuje uspešnost tehničnega pregleda glede na spremembo števila prevoženih kilometrov oz. glede na znamko avtomobila.




## ŠTEVILO TEHNIČNIH PREGLEDOV  IN ODSTOTEK USPEŠNO OPRAVLJENIH PREGLEDOV PO LETIH
Za vsako leto smo izračunali število tehničnih pregledov in nato še odstotek uspešnih pregledov in oboje hkrati prikazali na grafu.

Ugotovili smo, da je bilo največ tehničnih pregledov izvedenih leta 2021, opazili pa smo tudi padec leta 2020 zaradi vpliva COVID-19. Prav tako je bilo največ neuspešnih pregledov izvedenih v letu 2021.

## UVOZI VOZIL IN PRIMERJAVA MED UVOŽENIMI IN DOMESTIČNIMI VOZILI

Najprej smo za vsako leto izračunali delež uvoženih vozil in to prikazali na grafu.

Ugotovili smo, da se uvoz vozil z leti linearno zvišuje. Nato smo še primerjali, če imajo uvožena vozila večjo verjetnost, da opravijo tehnični pregled.


Ugotovili smo, da imajo uvožena vozila večjo verjetnost, da uspešno opravijo tehnični pregled.

## VOZILA S PREVRTENIMI KILOMETRI

Po letih smo analizirali prevožene kilometre posameznih vozil in si shranili tiste, ki so imeli na kasnejših tehničnih pregledih manj kilometrov kot na prejšnjih, kar pomeni, da so kilometre prevrteli nazaj.


Izpis programa nam pove, da je bilo v petih letih 1.32% vozil s prevrtenimi kilometri.

Nato smo te podatke uporabili v povezavi z drugimi atributi kot so znamka in namen vozila, da smo ugotovili, kateri znamki oz. vozilom s katerim namenom so največkrat prevrteli kilometre.





## IZVOZI VOZIL
Najprej smo za vsako leto izračunali delež izvoženih vozil in to prikazali na grafu. Za spremembo od uvoza, tu nismo opazili posebnega vzorca. 


Ugotovili smo tudi katera znamka je bila najbolj popularna za izvoz.

